# LangChain 1.0 Dynamic System Prompt

Ref:
https://docs.langchain.com/oss/python/langchain/agents#dynamic-system-prompt

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain.chat_models import init_chat_model
import mlflow

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = init_chat_model(
    "openai:databricks-gpt-oss-20b",
    api_key=creds.token,
    base_url=creds.host + "/serving-endpoints",
)


## エージェントの作成と実行

In [0]:
from typing import TypedDict
from langchain.agents.middleware import dynamic_prompt, ModelRequest

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """ユーザーの役割に基づいてシステムプロンプトを生成します。"""

    # 動作がわかるように、mlflowのtracingを利用
    with mlflow.start_span(name="user_role_prompt") as span:
        user_role = request.runtime.context.get("user_role", "user")
        base_prompt = "You are a helpful assistant."
        final_prompt = base_prompt

        span.set_inputs(user_role)

        if user_role == "expert":
            final_prompt = f"{base_prompt} Provide detailed technical responses."
        elif user_role == "beginner":
            final_prompt = f"{base_prompt} Explain concepts simply and avoid jargon."

        span.set_outputs(final_prompt)
        return final_prompt

In [0]:
from langchain.agents import create_agent
from pprint import pprint


def get_weather(city: str) -> str:
    """指定した都市の天気を取得します。"""
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=model,
    tools=[get_weather],
    middleware=[user_role_prompt],
    context_schema=Context,
)

input = {"messages": [{"role": "user", "content": "SFの天気は？"}]}
context = {"user_role": "expert"}
for event in agent.stream(input, context=context, stream_mode="updates"):
    pprint(event)